In [ ]:
# parameters initialization-----------------------------------------------------
import numpy as np
config_B = 400e6
config_L = 80  # number of slots
config_Lb = 4  # Bandwidth factor, in paper Lb ∈ [1,6]
config_Tc = 1 / config_B  # chip time, symbol duration ???
config_T = 2 * config_Lb * config_Tc  # Time slot duration: 2
config_beta = 0.25  # roll-off factor-(rrc) ???
config_t = np.linspace(
    0, config_L * config_T, config_L * 2 * config_Lb)  # Time vector: MaxTime: L * T = 80 * 1 = 80, No of samples: L * 2 * Lb = 80*2*2=320
config_x = np.random.binomial(n=1, p=0.5, size=config_L)  # np.random.randint(0, 2, L), x ∈ {0,1}
config_v = np.random.binomial(n=1, p=0.5)  # Target presence: Bernoulli(0.5)
config_upsample = 8

config_channel_duration = 1e-7
config_channel_samples = int(config_channel_duration // (config_Tc / (config_upsample * 2 * config_Lb)))

config_Nc = 5  # number of clutters
config_sigma_0 = 2.0  # Variance (or power) of the target amplitude β0 ???
config_tau_0 = np.random.uniform(0, 4 * config_Tc, 1)  # target delay
config_tau_c = np.random.uniform(0, 4 * config_Tc, config_Nc)  # np.random.choice(np.arange(0, int(4 * Tc)+1), size=Nc, replace=False) #np.random.uniform(0, 4 * Tc, Nc)  # clutter delays uniformly distributed between 0 and 4Tc

config_k_c = 2  # Shape parameter for Weibull distribution, κ ∈ [0.25,2]
config_lambda_c = 1.0  # Scale parameter for Weibull distribution, λ ∈ (0,∞)  ???

# total input data size based on paper: 60000(train) + 10000(test) = 70000



#Create datasets----------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.signal import convolve
# np.random.seed(42) #For having fixed values every time the code is run.


def generate_complex_gaussian(n, mean, variance):
    real_part = np.random.normal(mean, np.sqrt(variance / 2), n)
    imag_part = np.random.normal(mean, np.sqrt(variance / 2), n)
    return real_part + 1j * imag_part


def generate_s_t(t, T, L, x, Lb, Tc, beta):

    s_i = np.zeros(len(t), dtype=complex)

    # Loop in each time slot
    for l in range(len(x)):
        if x[l]:
            s_i[l * 2 * Lb + Lb] = 1.0
        else:
            s_i[l * 2 * Lb] = 1.0

    s_t_prime = np.zeros((len(s_i) * config_upsample), dtype=complex)
    s_t_prime[:: config_upsample] = s_i

    pulse_length = 101
    time = np.arange(pulse_length) - (pulse_length - 1) // 2
    Ts = config_upsample
    phi_t = (
        np.sinc(time / Ts)
        * np.cos(np.pi * beta * time / Ts)
        / (1 - (2 * beta * time / Ts) ** 2 + 1e-8)
    ).astype(complex)

    # plt.figure(1)
    # plt.stem(time, phi_t)
    # plt.grid()
    # plt.show()

    s_t = np.convolve(s_t_prime, phi_t, "same")

    # plt.figure()
    # plt.title("s(i)/s(t)")
    # plt.stem(Tc * np.arange(len(s_i)), s_i.real, label="s(i)", basefmt="b-")
    # plt.plot(
    #     (Tc / config_upsample) * np.arange(len(s_i) * config_upsample),
    #     s_t.real,
    #     "r",
    #     label="s(t)",
    # )
    # plt.legend(loc="upper right")
    # plt.show()

    return t, s_t


def channel_response(beta, Tc, v, sigma_0, tau_0, tau_c, Nc, beta_0, beta_c):

    # beta_0 = generate_complex_gaussian(1, mean=0, variance=1)
    # beta_c = generate_complex_gaussian(Nc, mean=0, variance=sigma_0)
    # print("B0: ", beta_0)
    # print("Bc: ", beta_c)

    h_t = np.zeros(config_channel_samples, dtype=complex)
    print("h_t len: ", len(h_t)) #2559

    upsampled_time = np.arange(len(h_t)) * (Tc / (2 * config_Lb * config_upsample)) #channel_duration = 1e-7
    print("upsampled_time len: ", len(upsampled_time)) #2559

    # generate the delays
    tau_0_dis = np.digitize(tau_0, upsampled_time) # find the index in upsampled_time that corresponds to the value of tau_0.
    tau_c_dis = np.digitize(tau_c, upsampled_time)
    print("tau_c_dis: ", tau_c_dis)

    h_t[tau_0_dis] = v * beta_0
    h_t[tau_c_dis] = beta_c

    # h_t = np.convolve(h_t, phi_rx, "same")

    # plt.plot((Tc / config_upsample) * np.arange(len(h_t)), np.abs(h_t), label= "channel")
    # plt.show()

    return h_t


def received_signal(s_t, h_t):

    y = convolve(s_t, h_t, mode="same")
    return y


def sample_received_signal(y, h, st):
    first_nonzero = np.argwhere(np.abs(h))[0][0]
    last_samples = y[-first_nonzero:]
    y = np.concatenate((y, last_samples))
    y_downsample = y[first_nonzero :: config_upsample]
    # plt.plot(np.abs(h))
    # plt.title(first_nonzero)
    # plt.show()

    print(f"len(y):{len(y)}, y_downsample: {len(y_downsample)}, first_nonzero = {first_nonzero}")

    yl = y_downsample.reshape(config_L, 2*config_Lb)
    print("yl(i) shape: ", yl.shape)
    yl_mapped = complex_to_real_vector(yl)
    print("yl_mapped shape: ", yl_mapped.shape)


    xl = st[ :: config_upsample].reshape(config_L, 2*config_Lb)
    print("xl(i) shape: ", xl.shape)
    xl_mapped = complex_to_real_vector(xl)
    print("xl_mapped shape: ", xl_mapped.shape)

    return yl_mapped, xl_mapped


def complex_to_real_vector(yl):
    y_mapped=[]
    for row in yl:
        real_part = row.real
        imag_part = row.imag
        row_concat = np.concatenate((real_part,imag_part))

        y_mapped.append(row_concat)

    return np.array(y_mapped)


def create_noise(num_noise_samples, h_t):
    Eb = 1
    SNR = 10  # SNR is given 10 dB
    squared_norm_h = np.abs(h_t) ** 2  #|h|^2
    average_squared_norm_h = np.mean(squared_norm_h)
    N0B = (average_squared_norm_h * Eb) / SNR
    # print(f"_____N0B: {N0B}")
    noise = np.random.normal(0, np.sqrt(N0B), num_noise_samples) + 1j * np.random.normal(0, np.sqrt(N0B), num_noise_samples)
    return noise


#------------------main------------------
num_round = 100000
multiple_tuples_1=[]
multiple_tuples_2=[]

for r in range(num_round):
    x = np.random.binomial(n=1, p=0.5, size=config_L)  # np.random.randint(0, 2, L), x ∈ {0,1}
    v = np.random.binomial(n=1, p=0.5)  # Target presence: Bernoulli(0.5)
    tau_0 = np.array([0])
    if v==1:
        tau_0 = np.random.uniform(0, 4 * config_Tc, 1)  # target delay
    tau_label = tau_0[0]

    # Create signals----------
    t, s_t = generate_s_t(
        config_t, config_T, config_L, x, config_Lb, config_Tc, config_beta
    )
    print("length of s(t):", len(s_t))


    beta_0 = generate_complex_gaussian(1, mean=0, variance=1)
    beta_c = generate_complex_gaussian(config_Nc, mean=0, variance=config_sigma_0)
    print("B0: ", beta_0)
    print("Bc: ", beta_c)

    #First channel---------------
    h1_t = channel_response(
        config_beta,
        config_Tc,
        v,
        config_sigma_0,
        config_tau_0,
        config_tau_c,
        config_Nc,
        beta_0,
        beta_c,
    )

    y1_t = received_signal(s_t, h1_t)
    print("length of y(t):", len(y1_t))

    # Add noise to y(t)---
    z1 = create_noise(len(y1_t), h1_t)
    y1_noisy = y1_t + z1

    # Downsample the y1(t)---
    yl1, xl1 = sample_received_signal(y1_noisy, h1_t, s_t)
    multiple_tuples_1.append((yl1,xl1,v))

    #Second channel---------------
    h2_t = channel_response(
        config_beta,
        config_Tc,
        v,
        config_sigma_0,
        tau_0,
        config_tau_c,
        config_Nc,
        beta_0,
        beta_c,
    )

    y2_t = received_signal(s_t, h2_t)
    print("length of y(t):", len(y2_t))

    # Add noise to y(t)--------
    z2 = create_noise(len(y2_t), h2_t)
    y2_noisy = y2_t + z2

    # Downsample the y1(t)------
    yl2, xl2 = sample_received_signal(y2_noisy, h2_t, s_t)
    print("round# :", r, " v : ", v , " tau:", tau_label)
    multiple_tuples_2.append((yl2, xl2, v, tau_label))



    # Checks both values and shape----------
    if np.array_equal(xl1, xl2):
      print("✅ x1 and x2 are exactly equal")
    else:
      print("❌ x1 and x2 are NOT equal")



    # Print one sample
    if r == 1:
        plt.plot((config_Tc / config_upsample) * np.arange(len(s_t)), s_t.real, label = "s_t")
        plt.plot((config_Tc / config_upsample) * np.arange(len(h1_t)), np.abs(h1_t), label="h_t")
        plt.plot((config_Tc / config_upsample) * np.arange(len(y1_t)), y1_t.real, label="y_t")
        plt.legend()
        plt.show()

        plt.figure(figsize=(10, 5))
        plt.subplot(2,1,1)
        plt.plot(config_Tc * np.arange(len(y1_noisy)), y1_noisy.real, label="y_noisy", color='r')
        plt.legend()
        plt.subplot(2,1,2)
        plt.plot(config_Tc * np.arange(len(yl1[:, :8].real.flatten())), yl1[:, :8].real.flatten(), label="y_downSampled", color='b')
        plt.legend()
        plt.show()



In [ ]:
import pickle

#Create dataset---------------
with open("/content/drive/MyDrive/Colab Notebooks/SNN/Dataset/dataset5.pkl", "wb") as f:
    pickle.dump(multiple_tuples_1, f)

print("Dataset5 is generated!")

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/SNN/Dataset/dataset4_Tau.pkl", "wb") as f:
    pickle.dump(multiple_tuples_2, f)

print("Dataset4_Tau is generated!")

In [3]:
import plotly.graph_objects as go
import pickle

#Load datasets and show on sampel
with open("/content/drive/MyDrive/Colab Notebooks/SNN/Dataset/dataset5.pkl", "rb") as f:
    multiple_tuples_1 = pickle.load(f)

with open("/content/drive/MyDrive/Colab Notebooks/SNN/Dataset/dataset5_Tau.pkl", "rb") as f:
    multiple_tuples_2 = pickle.load(f)


#plot dataset----
y_test, x_test, v_test = multiple_tuples_1[3]
y_test_real = y_test[:, :8].real.flatten()
x_test_real = x_test[:, :8].flatten()

fig = go.Figure()
fig.add_trace(go.Scatter(y = y_test_real, mode='lines', name="y"))
# # Stem plot for `x_test_real`
# fig.add_trace(go.Scatter(y=x_test_real, mode='markers', marker=dict(color='red', size=8), name="x"))
fig.update_layout(title=f"multiple_tuples, v = {v_test}",
                  xaxis_title="Index",
                  yaxis_title="Value",
                  hovermode="x unified")
fig.show()


#plot dataset----
y_test2, x_test2, v_test2,_ = multiple_tuples_2[3]
y_test_real2 = y_test2[:, :8].real.flatten()
x_test_real2 = x_test2[:, :8].flatten()

fig = go.Figure()
fig.add_trace(go.Scatter(y = y_test_real2, mode='lines', name="y"))
# # Stem plot for `x_test_real`
# fig.add_trace(go.Scatter(y=x_test_real2, mode='markers', marker=dict(color='red', size=8), name="x"))
fig.update_layout(title=f"multiple_tuples_Tau, v = {v_test}",
                  xaxis_title="Index",
                  yaxis_title="Value",
                  hovermode="x unified")
fig.show()




# Class distribution
v_data1 = [sample[2] for sample in multiple_tuples_1]
v_data2 = [sample[2] for sample in multiple_tuples_2]

num_ones = sum(v_data1)  # Counts all 1s
num_zeros = len(v_data1) - num_ones  # Remaining are 0s

print(f"Number of 0s: {num_zeros}")
print(f"Number of 1s: {num_ones}")

Number of 0s: 50068
Number of 1s: 49932
